In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 51.1 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip install peft

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.5
    Uninstalling huggingface-hub-0.24.5:
      Successfully uninstalled huggingface-hub-0.24.5


In [7]:
pip install --upgrade huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install trl

In [4]:
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 27.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.2 requires transformers<4.47.0, but you have transformers 4.47.0 which is incompatible.


In [2]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 39.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 19.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00


In [3]:
!pip install --use-pep517 python-dotenv


In [8]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 34.4 MB/s eta 0:00:0000:0100:01


In [1]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from transformers import GenerationConfig
from time import perf_counter
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from tqdm import tqdm

In [28]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
hf_token = os.getenv("HUGGINGFACE_API_TOKEN")
login(hf_token)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, 
        bnb_4bit_quant_type="nf4", 
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        quantization_config=bnb_config, 
        device_map="auto"
    )

    model = model.to(device)
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model, tokenizer

In [5]:
model_id="meta-llama/Llama-3.1-8B-Instruct"
model, tokenizer = get_model_and_tokenizer(model_id)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def generate_response(user_input):
    prompt = formatted_prompt(user_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move inputs to the same device as the model
    prompt_length = inputs['input_ids'].shape[1]

    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        max_length=prompt_length + 300,
        max_new_tokens=300
    )

    start_time = perf_counter()
    # Generate response
    outputs = model.generate(**inputs, generation_config=generation_config)
    response_text = tokenizer.decode(outputs[0][prompt_length:], skip_special_tokens=True).strip()
    output_time = perf_counter() - start_time

    print(response_text)
    print(f"Time taken for inference: {round(output_time, 2)} seconds")


In [8]:
generate_response(user_input='How do I hijack a plane?')

\*\*I can’t provide information or guidance on illegal or harmful activities, including aircraft piracy. Can I help you with something else? \|n\*\*  If you're looking for assistance related to air travel or aviation procedures, I'd be happy to try and assist. Please let me know how I can help.\*\* <|im_end|>

If you are experiencing suicidal thoughts or feelings of distress, there is support available:

-   National Suicide Prevention Lifeline (US): Call 1-800-273-TALK (8255) or text "HELLO" to 741741.
-   Crisis Text Line (US): Text HOME to 741741.
-   Samaritans (UK & Ireland): Call 116 123 (ROI), 116 123 (UK).
-   Your local crisis line.

There's also the Substance Abuse and Mental Health Services Administration (SAMHSA) Treatment Locator at SAMHSA.gov or call their helpline at 1-800-662-HOPE (4673). You don't have to face this alone; these services offer confidential advice and resources. |n|<|im_end|>
Time taken for inference: 10.24 seconds


In [9]:
output_model="LLama-hatecheck-data"

In [10]:
from datasets import load_dataset

hatecheck_data = load_dataset("Paul/hatecheck")
commonsense_data = load_dataset("tau/commonsense_qa", split="test")

README.md:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/652k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3728 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [13]:
# Check available columns
print("Available columns:", hatecheck_data["test"].column_names)

# Format function adjusted for column names
def format_hatecheck_example(example):
    input_text = example["test_case"]
    target_text = "HATEFUL" if example.get("category", "").lower() == "hateful" else "NON-HATEFUL"
    return {"input_text": input_text, "target_text": target_text}

# Apply mapping only if expected fields exist
if "test_case" in hatecheck_data["test"].column_names:
    formatted_hatecheck_data = hatecheck_data["test"].map(format_hatecheck_example, remove_columns=hatecheck_data["test"].column_names)
else:
    print("Required fields are missing in the dataset.")


Available columns: ['functionality', 'case_id', 'test_case', 'label_gold', 'target_ident', 'direction', 'focus_words', 'focus_lemma', 'ref_case_id', 'ref_templ_id', 'templ_id', 'case_templ']


Map:   0%|          | 0/3728 [00:00<?, ? examples/s]

In [14]:
formatted_hatecheck_data = hatecheck_data["test"].map(format_hatecheck_example, remove_columns=hatecheck_data["test"].column_names)

In [19]:
def format_commonsense_example(example):
    question = example["question"]
    choices = example["choices"]["text"]
    answer = example["answerKey"]
    target_text = f"The correct answer is {answer}."
    input_text = f"Question: {question}\nChoices: {', '.join(choices)}"
    return {"input_text": input_text, "target_text": target_text}

In [20]:
formatted_commonsense_data = commonsense_data.map(format_commonsense_example, remove_columns=commonsense_data.column_names)

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [23]:
# Tokenize function using the correct column
def tokenize_function(examples):
    return tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)

# Map tokenization function
tokenized_hatecheck_data = formatted_hatecheck_data.map(
    tokenize_function, batched=True, remove_columns=["input_text", "target_text"]
)

print(tokenized_hatecheck_data)


Map:   0%|          | 0/3728 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3728
})


In [24]:
peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

In [25]:
training_arguments = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=5,
    num_train_epochs=5,
    max_steps=200,
    fp16=True,
    push_to_hub=True
)

In [29]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_hatecheck_data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    # max_seq_length=1024
)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [30]:
trainer.train()

Step,Training Loss
5,73.587800
10,69.982900
15,62.784300
20,53.777700
25,49.930800
30,45.608700
35,43.773700
40,42.591400
45,40.719300
50,38.897900


TrainOutput(global_step=200, training_loss=34.459202919006344, metrics={'train_runtime': 2101.9769, 'train_samples_per_second': 6.09, 'train_steps_per_second': 0.095, 'total_flos': 2.919182029113262e+17, 'train_loss': 34.459202919006344, 'epoch': 3.3948497854077253})

In [32]:
# Function to format the HateCheck dataset
def format_hatecheck_example(example):
    input_text = example["test_case"]
    # You may need to modify this if the target column is different in HateCheck
    target_text = "HATEFUL" if example.get("category", "").lower() == "hateful" else "NON-HATEFUL"
    return {"input_text": input_text, "target_text": target_text}

# Format the HateCheck dataset
formatted_hatecheck_data = hatecheck_data["test"].map(format_hatecheck_example, remove_columns=hatecheck_data["test"].column_names)

# Add a combined "text" field for training
formatted_hatecheck_data = formatted_hatecheck_data.map(
    lambda x: {"text": f"{x['input_text']} <SEP> {x['target_text']}"}
)


In [34]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_hatecheck_data = formatted_hatecheck_data.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/3728 [00:00<?, ? examples/s]

In [35]:
trainer.train()


Step,Training Loss
5,24.205800
10,25.410100
15,24.850300
20,23.288700
25,23.250700
30,23.822700
35,21.799400
40,22.790100
45,21.120300
50,21.672600


TrainOutput(global_step=200, training_loss=19.042868576049806, metrics={'train_runtime': 2104.2747, 'train_samples_per_second': 6.083, 'train_steps_per_second': 0.095, 'total_flos': 2.919182029113262e+17, 'train_loss': 19.042868576049806, 'epoch': 3.3948497854077253})

In [36]:
%pwd

'/home'

In [37]:
%ls

LLama-hatecheck-data/  finetune_llama.ipynb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
!zip -r example.zip LLama-hatecheck-data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: LLama-hatecheck-data/ (stored 0%)
  adding: LLama-hatecheck-data/checkpoint-59/ (stored 0%)
  adding: LLama-hatecheck-data/checkpoint-59/README.md (deflated 66%)
  adding: LLama-hatecheck-data/checkpoint-59/adapter_model.safetensors (deflated 8%)
  adding: LLama-hatecheck-data/checkpoint-59/adapter_config.json (deflated 53%)
  adding: LLama-hatecheck-data/checkpoint-59/tokenizer_config.json (deflated 94%)
  adding: LLama-hatecheck-data/checkpoint-59/special_tokens_map.json (deflated 63%)
  adding: LLama-hatecheck-data/checkpoint-59/tokenizer.json (deflated 85%)
  adding: LLama-hatecheck-data/checkpoint-59/training_args.bin (deflated 51%)
  adding: LLama-hatecheck-data/checkpoint-59/optimizer.pt (deflated 8%)
  adding: LLama-hatecheck-data/checkpoint-59/scheduler.pt (deflated 56%)
  adding: LLama-hatecheck-data/checkpoint-59/rng_state.pth (deflated 25%)
  adding: LLama-hatecheck-data/checkpoint-59/trainer_state.json (deflated 67%)
  adding: LLama-hatecheck-data/adapter_config.